In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from tkinter import Tk, filedialog
import sys
import pandas as pd

# --- Configuración inicial ---
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

# --- Función para seleccionar video ---
print("sSeleccione el video:")
def select_video():
    root = Tk()
    root.withdraw()
    video_path = filedialog.askopenfilename(
        title="Selecciona el video ecocardiográfico",
        filetypes=[("Videos", "*.avi *.mp4 *.mov"), ("Todos los archivos", "*.*")]
    )
    root.destroy()
    return video_path

# --- Cargar video ---
video_path = select_video()
if not video_path:
    print("No se seleccionó ningún archivo.")
    sys.exit()

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error al abrir el video. Verifica:")
    print(f"- Ruta: {video_path}")
    print("- Formato (convierte a MP4 si es necesario)")
    print("- Instala codecs con: pip install ffmpeg-python")
    sys.exit()

fps = cap.get(cv2.CAP_PROP_FPS)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"\nVideo cargado: {video_path}")
print(f"Resolución: {int(cap.get(3))}x{int(cap.get(4))}")
print(f"FPS: {fps:.2f}, Total frames: {num_frames}")

# --- Selección manual de puntos ---
ret, frame1 = cap.read()
if not ret:
    print("Error al leer el primer frame.")
    sys.exit()

frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Configurar matplotlib para modo interactivo
plt.switch_backend('TkAgg')  # Usar backend interactivo
plt.imshow(frame1_gray, cmap='gray')
plt.title("Seleccione 2 puntos:\n1. Pared septal (Click 1)\n2. Pared lateral (Click 2)")
print("\nPor favor, seleccione 2 puntos en la imagen...")
points = plt.ginput(2, timeout=0)
plt.close()

if len(points) < 2:
    print("Error: Debes seleccionar exactamente 2 puntos.")
    sys.exit()

x1, y1 = int(points[0][0]), int(points[0][1])
x2, y2 = int(points[1][0]), int(points[1][1])
roi_size = 10  # Tamaño del ROI (20x20 píxeles)

print(f"\nPuntos seleccionados:")
print(f"- Pared septal: ({x1}, {y1})")
print(f"- Pared lateral: ({x2}, {y2})")

# --- Extraer parches de referencia ---
patch1 = frame1_gray[y1-roi_size:y1+roi_size, x1-roi_size:x1+roi_size]
patch2 = frame1_gray[y2-roi_size:y2+roi_size, x2-roi_size:x2+roi_size]

# Verificar que los parches sean válidos
if patch1.size == 0 or patch2.size == 0:
    print("Error: Los puntos seleccionados están demasiado cerca del borde.")
    sys.exit()

# --- Seguimiento de puntos ---
displacements1, displacements2 = [y1], [y2]

print("\nProcesando video...")
for i in range(1, num_frames):
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Correlación cruzada normalizada
    c1 = cv2.matchTemplate(gray_frame, patch1, cv2.TM_CCOEFF_NORMED)
    c2 = cv2.matchTemplate(gray_frame, patch2, cv2.TM_CCOEFF_NORMED)

    # Posición del máximo
    _, max_val1, _, max_loc1 = cv2.minMaxLoc(c1)
    _, max_val2, _, max_loc2 = cv2.minMaxLoc(c2)

    # Umbral de confianza (0.5)
    if max_val1 > 0.5 and max_val2 > 0.5:
        displacements1.append(max_loc1[1] + roi_size)
        displacements2.append(max_loc2[1] + roi_size)
    else:
        displacements1.append(displacements1[-1])
        displacements2.append(displacements2[-1])

cap.release()

# --- Cálculo de deformación y strain rate ---
displacements1 = np.array(displacements1)
displacements2 = np.array(displacements2)

defor1 = (displacements1 - displacements1[0]) / displacements1[0]
defor2 = (displacements2 - displacements2[0]) / displacements2[0]

dt = 1 / fps
strain_rate1 = np.diff(defor1) / dt
strain_rate2 = np.diff(defor2) / dt

# --- Resultados ---
print("\n--- Resultados ---")
print(f"Deformación máxima (Septal): {np.min(defor1):.4f}")  # Normalmente negativo
print(f"Deformación máxima (Lateral): {np.min(defor2):.4f}")
print(f"Strain Rate máximo (Septal): {np.min(strain_rate1):.2f} 1/s")
print(f"Strain Rate máximo (Lateral): {np.min(strain_rate2):.2f} 1/s")

# --- Gráficos ---
plt.figure(figsize=(12, 8))

# Gráfico de deformación
plt.subplot(2, 1, 1)
plt.plot(defor1, 'r', label='Septal', linewidth=2)
plt.plot(defor2, 'b', label='Lateral', linewidth=2)
plt.title("Deformación Normalizada (Strain)")
plt.xlabel("Frame")
plt.ylabel("Strain")
plt.legend()
plt.grid()

# Gráfico de strain rate
plt.subplot(2, 1, 2)
time = np.arange(len(strain_rate1)) * dt
plt.plot(time, strain_rate1, 'r', label='Septal', linewidth=2)
plt.plot(time, strain_rate2, 'b', label='Lateral', linewidth=2)
plt.title("Tasa de Deformación (Strain Rate)")
plt.xlabel("Tiempo (s)")
plt.ylabel("1/s")
plt.legend()
plt.grid()

plt.tight_layout()

# --- Guardar resultados ---
df = pd.DataFrame({
    'Frame': range(len(defor1)),
    'Strain_Septal': defor1,
    'Strain_Lateral': defor2,
    'Strain_Rate_Septal': np.append(strain_rate1, np.nan),  # Ajustar longitud
    'Strain_Rate_Lateral': np.append(strain_rate2, np.nan)
})


sSeleccione el video:

Video cargado: C:/Users/geron/OneDrive/Escritorio/documentos/Eko/0X1A296F5FCD5A0ED8.avi
Resolución: 112x112
FPS: 50.00, Total frames: 131

Por favor, seleccione 2 puntos en la imagen...

Puntos seleccionados:
- Pared septal: (58, 78)
- Pared lateral: (90, 70)

Procesando video...

--- Resultados ---
Deformación máxima (Septal): -0.3846
Deformación máxima (Lateral): -0.8286
Strain Rate máximo (Septal): -23.72 1/s
Strain Rate máximo (Lateral): -57.14 1/s
